In [ ]:
import numpy as np
import pandas as pd
import cv2
import requests
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns
import os
from urllib.parse import urlparse

# Text processing libraries
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Image processing libraries
from skimage import feature, color, measure, filters, segmentation
from scipy.stats import skew, kurtosis
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('vader_lexicon')
except LookupError:
    print("Downloading NLTK data...")
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('vader_lexicon')

class ImageSentimentAnalyzer:
    """Comprehensive image sentiment analyzer using computer vision technique """

    def __init__(self):
        # Initialize deep learning model for feature extraction
        try:
            self.image_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
            self.deep_learning_available = True
        except:
            print("Warning: MobileNetV2 not available, using basic image analysis only")
            self.deep_learning_available = False

        # Color emotion mapping based on color psychology research
        self.color_emotions = {
            'red': {'arousal': 0.8, 'valence': 0.2, 'dominance': 0.7},      # High arousal, negative valence
            'orange': {'arousal': 0.7, 'valence': 0.6, 'dominance': 0.6},   # Energetic, positive
            'yellow': {'arousal': 0.6, 'valence': 0.8, 'dominance': 0.5},   # Happy, bright
            'green': {'arousal': 0.3, 'valence': 0.6, 'dominance': 0.4},    # Calm, natural
            'blue': {'arousal': 0.2, 'valence': 0.4, 'dominance': 0.3},     # Calm, sometimes sad
            'purple': {'arousal': 0.4, 'valence': 0.5, 'dominance': 0.6},   # Mysterious, creative
            'pink': {'arousal': 0.5, 'valence': 0.7, 'dominance': 0.3},     # Gentle, loving
            'brown': {'arousal': 0.3, 'valence': 0.4, 'dominance': 0.4},    # Natural, stable
            'black': {'arousal': 0.4, 'valence': 0.1, 'dominance': 0.8},    # Strong, sometimes negative
            'white': {'arousal': 0.2, 'valence': 0.6, 'dominance': 0.3},    # Pure, clean
            'gray': {'arousal': 0.1, 'valence': 0.3, 'dominance': 0.2}      # Neutral, sometimes sad
        }

        # Facial expression patterns (simplified)
        self.expression_patterns = {
            'smile_curve': 'positive',
            'frown_curve': 'negative',
            'bright_eyes': 'positive',
            'dark_shadows': 'negative'
        }

    def load_image(self, image_input):
        """
        Load image from various input types (file path, URL, PIL Image)
        """
        try:
            if isinstance(image_input, str):
                if image_input.startswith(('http://', 'https://')):
                    # You can load from URL
                    response = requests.get(image_input, timeout=10)
                    response.raise_for_status()
                    img = Image.open(BytesIO(response.content))
                else:
                    # Load from file path
                    if not os.path.exists(image_input):
                        raise FileNotFoundError(f"Image file not found: {image_input}")
                    img = Image.open(image_input)
            elif isinstance(image_input, Image.Image):
                img = image_input
            else:
                raise ValueError("Unsupported image input type")

            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')

            return img

        except Exception as e:
            print(f"Error loading image: {e}")
            return None

    def analyze_color_sentiment(self, img):
        """
        Analyze sentiment based on color composition and psychology
        """
        img_array = np.array(img)
        height, width = img_array.shape[:2]

        # Convert to different color spaces
        hsv_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
        lab_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2LAB)

        # Extract color features
        features = {}

        # 1. Dominant colors analysis
        pixels = img_array.reshape(-1, 3)

        # Calculate color dominance
        r_mean, g_mean, b_mean = np.mean(pixels, axis=0)
        r_std, g_std, b_std = np.std(pixels, axis=0)

        features['red_dominance'] = r_mean / 255.0
        features['green_dominance'] = g_mean / 255.0
        features['blue_dominance'] = b_mean / 255.0
        features['color_variance'] = np.mean([r_std, g_std, b_std]) / 255.0

        # 2. HSV analysis
        hue = hsv_img[:, :, 0]
        saturation = hsv_img[:, :, 1]
        value = hsv_img[:, :, 2]

        features['average_hue'] = np.mean(hue)
        features['average_saturation'] = np.mean(saturation) / 255.0
        features['average_brightness'] = np.mean(value) / 255.0
        features['saturation_variance'] = np.std(saturation) / 255.0

        # 3. Color temperature (warm vs cool)
        warm_pixels = np.sum((pixels[:, 0] > pixels[:, 2]) & (pixels[:, 1] > pixels[:, 2]))
        cool_pixels = np.sum((pixels[:, 2] > pixels[:, 0]) & (pixels[:, 2] > pixels[:, 1]))
        total_pixels = len(pixels)

        features['warmth_ratio'] = warm_pixels / total_pixels
        features['coolness_ratio'] = cool_pixels / total_pixels

        # 4. Color harmony analysis
        hue_std = np.std(hue)
        features['color_harmony'] = 1.0 / (1.0 + hue_std / 180.0)  # Higher = more harmonious

        # 5. Map to emotional dimensions
        sentiment_score = 0
        confidence_factors = []

        # Brightness sentiment
        if features['average_brightness'] > 0.7:
            sentiment_score += 0.3  # Bright = positive
            confidence_factors.append(0.6)
        elif features['average_brightness'] < 0.3:
            sentiment_score -= 0.2  # Dark = negative
            confidence_factors.append(0.5)

        # Saturation sentiment
        if features['average_saturation'] > 0.6:
            sentiment_score += 0.2  # Vibrant = positive
            confidence_factors.append(0.4)
        elif features['average_saturation'] < 0.3:
            sentiment_score -= 0.1  # Dull = less positive

        # Warmth sentiment
        if features['warmth_ratio'] > 0.6:
            sentiment_score += 0.2  # Warm = positive
            confidence_factors.append(0.3)
        elif features['coolness_ratio'] > 0.6:
            sentiment_score -= 0.1  # Cool = less positive

        # Color harmony sentiment
        if features['color_harmony'] > 0.7:
            sentiment_score += 0.1  # Harmonious = positive

        return {
            'color_features': features,
            'sentiment_score': sentiment_score,
            'confidence': min(0.8, np.mean(confidence_factors) if confidence_factors else 0.3)
        }

    def analyze_texture_sentiment(self, img):
        """
        Analyze sentiment based on texture and patterns
        """
        img_array = np.array(img)
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

        features = {}

        # 1. Edge density analysis
        edges = cv2.Canny(gray, 50, 150)
        edge_density = np.sum(edges > 0) / (gray.shape[0] * gray.shape[1])
        features['edge_density'] = edge_density

        # 2. Texture smoothness
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        features['texture_roughness'] = laplacian_var / 10000.0  # Normalize

        # 3. Local Binary Pattern analysis
        try:
            from skimage import feature as sk_feature
            lbp = sk_feature.local_binary_pattern(gray, P=8, R=1, method='uniform')
            lbp_hist, _ = np.histogram(lbp.ravel(), bins=10, range=(0, 9))
            lbp_hist = lbp_hist.astype(float)
            lbp_hist /= (lbp_hist.sum() + 1e-7)

            # Texture uniformity
            features['texture_uniformity'] = -np.sum(lbp_hist * np.log2(lbp_hist + 1e-7))
        except:
            features['texture_uniformity'] = 0.5  # Default value

        # 4. Gradient analysis
        grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
        features['gradient_strength'] = np.mean(gradient_magnitude) / 255.0

        # 5. Contrast analysis
        features['contrast'] = np.std(gray) / 255.0

        # Sentiment mapping
        sentiment_score = 0
        confidence_factors = []

        # Smooth textures often more pleasant
        if features['texture_roughness'] < 0.3:
            sentiment_score += 0.1
            confidence_factors.append(0.3)
        elif features['texture_roughness'] > 0.7:
            sentiment_score -= 0.1
            confidence_factors.append(0.3)

        # Moderate contrast is pleasant
        if 0.3 <= features['contrast'] <= 0.7:
            sentiment_score += 0.1
            confidence_factors.append(0.4)

        # High edge density might indicate chaos (negative)
        if features['edge_density'] > 0.3:
            sentiment_score -= 0.1
            confidence_factors.append(0.2)

        return {
            'texture_features': features,
            'sentiment_score': sentiment_score,
            'confidence': min(0.6, np.mean(confidence_factors) if confidence_factors else 0.2)
        }

    def analyze_composition_sentiment(self, img):
        """
        Analyze sentiment based on image composition and layout
        """
        img_array = np.array(img)
        height, width = img_array.shape[:2]

        features = {}

        # 1. Rule of thirds analysis
        third_h, third_w = height // 3, width // 3

        # Calculate interest points at rule of thirds intersections
        interest_points = [
            (third_w, third_h), (2 * third_w, third_h),
            (third_w, 2 * third_h), (2 * third_w, 2 * third_h)
        ]

        # 2. Center vs edge brightness
        center_region = img_array[height//4:3*height//4, width//4:3*width//4]
        edge_regions = np.concatenate([
            img_array[:height//4, :].flatten(),  # Top
            img_array[3*height//4:, :].flatten(),  # Bottom
            img_array[:, :width//4].flatten(),   # Left
            img_array[:, 3*width//4:].flatten()  # Right
        ])

        center_brightness = np.mean(center_region)
        edge_brightness = np.mean(edge_regions)
        features['center_focus'] = center_brightness / (edge_brightness + 1)

        # 3. Symmetry analysis
        left_half = img_array[:, :width//2]
        right_half = np.fliplr(img_array[:, width//2:])

        # Resize to match if needed
        min_width = min(left_half.shape[1], right_half.shape[1])
        left_half = left_half[:, :min_width]
        right_half = right_half[:, :min_width]

        symmetry_score = np.mean(np.abs(left_half.astype(float) - right_half.astype(float)))
        features['horizontal_symmetry'] = 1.0 / (1.0 + symmetry_score / 255.0)

        # 4. Golden ratio analysis (approximation)
        golden_ratio = 1.618
        golden_y = int(height / golden_ratio)
        golden_x = int(width / golden_ratio)

        # Check if interesting features align with golden ratio
        features['golden_ratio_alignment'] = 0.5  # Simplified for demo

        # Sentiment mapping
        sentiment_score = 0
        confidence_factors = []

        # Good composition often positive
        if features['center_focus'] > 1.2:  # Center brighter than edges
            sentiment_score += 0.15
            confidence_factors.append(0.4)

        if features['horizontal_symmetry'] > 0.7:  # Good symmetry
            sentiment_score += 0.1
            confidence_factors.append(0.3)

        return {
            'composition_features': features,
            'sentiment_score': sentiment_score,
            'confidence': min(0.5, np.mean(confidence_factors) if confidence_factors else 0.2)
        }

    def analyze_deep_features(self, img):
        """
        Extract deep learning features if available
        """
        if not self.deep_learning_available:
            return {
                'deep_features': {},
                'sentiment_score': 0,
                'confidence': 0
            }

        try:
            # Preprocess image for MobileNetV2
            img_resized = img.resize((224, 224))
            img_array = np.array(img_resized)
            img_expanded = np.expand_dims(img_array, axis=0)
            img_preprocessed = preprocess_input(img_expanded)

            # Extract features
            deep_features = self.image_model.predict(img_preprocessed, verbose=0)

            features = {
                'deep_feature_mean': np.mean(deep_features),
                'deep_feature_std': np.std(deep_features),
                'deep_feature_max': np.max(deep_features),
                'deep_feature_min': np.min(deep_features),
                'deep_feature_energy': np.sum(deep_features ** 2)
            }

            # Simple sentiment mapping based on feature statistics
            sentiment_score = 0
            if features['deep_feature_mean'] > 0:
                sentiment_score += 0.1
            if features['deep_feature_energy'] > np.median(deep_features):
                sentiment_score += 0.05

            return {
                'deep_features': features,
                'sentiment_score': sentiment_score,
                'confidence': 0.3
            }

        except Exception as e:
            print(f"Error in deep feature extraction: {e}")
            return {
                'deep_features': {},
                'sentiment_score': 0,
                'confidence': 0
            }

    def analyze_image_sentiment(self, image_input):
        """
        Comprehensive image sentiment analysis
        """
        # Load image
        img = self.load_image(image_input)
        if img is None:
            return {
                'sentiment': 'Neutral',
                'confidence': 0.0,
                'error': 'Failed to load image'
            }

        print(f"Analyzing image: {img.size}")

        # Perform different types of analysis
        color_analysis = self.analyze_color_sentiment(img)
        texture_analysis = self.analyze_texture_sentiment(img)
        composition_analysis = self.analyze_composition_sentiment(img)
        deep_analysis = self.analyze_deep_features(img)

        # Combine all sentiment scores
        total_score = (
            color_analysis['sentiment_score'] * 0.4 +
            texture_analysis['sentiment_score'] * 0.3 +
            composition_analysis['sentiment_score'] * 0.2 +
            deep_analysis['sentiment_score'] * 0.1
        )

        # Combine confidences
        total_confidence = (
            color_analysis['confidence'] * 0.4 +
            texture_analysis['confidence'] * 0.3 +
            composition_analysis['confidence'] * 0.2 +
            deep_analysis['confidence'] * 0.1
        )

        # Determine final sentiment
        if total_score > 0.15:
            sentiment = 'Positive'
            confidence = min(0.9, 0.6 + total_confidence)
        elif total_score < -0.15:
            sentiment = 'Negative'
            confidence = min(0.9, 0.6 + total_confidence)
        else:
            sentiment = 'Neutral'
            confidence = max(0.3, total_confidence)

        return {
            'sentiment': sentiment,
            'confidence': confidence,
            'total_score': total_score,
            'analysis_breakdown': {
                'color_analysis': color_analysis,
                'texture_analysis': texture_analysis,
                'composition_analysis': composition_analysis,
                'deep_analysis': deep_analysis
            },
            'image_properties': {
                'size': img.size,
                'mode': img.mode
            }
        }

class MultimodalSentimentAnalyzer:
    """
    Multimodal sentiment analyzer combining text and image analysis
    """

    def __init__(self):
        # Initialize text sentiment analyzers
        self.vader_analyzer = SentimentIntensityAnalyzer()

        # Initialize image analyzer
        self.image_analyzer = ImageSentimentAnalyzer()

    def analyze_text_sentiment(self, text):
        """
        Analyze text sentiment using multiple approaches
        """
        if not text or text.strip() == "":
            return {
                'sentiment': 'Neutral',
                'confidence': 0.0,
                'scores': {'positive': 0, 'negative': 0, 'neutral': 1}
            }

        # Method 1: TextBlob sentiment analysis
        blob = TextBlob(text)
        textblob_polarity = blob.sentiment.polarity
        textblob_subjectivity = blob.sentiment.subjectivity

        # Method 2: VADER sentiment analysis
        vader_scores = self.vader_analyzer.polarity_scores(text)

        # Combine both methods
        combined_positive = (textblob_polarity > 0.1) or (vader_scores['pos'] > 0.3)
        combined_negative = (textblob_polarity < -0.1) or (vader_scores['neg'] > 0.3)

        # Determine final sentiment
        if combined_positive and not combined_negative:
            sentiment = 'Positive'
            confidence = min(0.95, abs(textblob_polarity) * 0.5 + vader_scores['pos'])
        elif combined_negative and not combined_positive:
            sentiment = 'Negative'
            confidence = min(0.95, abs(textblob_polarity) * 0.5 + vader_scores['neg'])
        else:
            sentiment = 'Neutral'
            confidence = max(vader_scores['neu'], 0.5)

        return {
            'sentiment': sentiment,
            'confidence': confidence,
            'scores': {
                'positive': vader_scores['pos'],
                'negative': vader_scores['neg'],
                'neutral': vader_scores['neu']
            },
            'textblob_polarity': textblob_polarity,
            'textblob_subjectivity': textblob_subjectivity
        }

    def analyze_multimodal_sentiment(self, text=None, image_path=None, text_weight=0.6, image_weight=0.4):
        """
        Combine text and image sentiment analysis
        """
        results = {
            'combined_sentiment': 'Neutral',
            'confidence': 0.0,
            'modalities_used': []
        }

        text_result = None
        image_result = None

        # Analyze text if provided
        if text:
            text_result = self.analyze_text_sentiment(text)
            results['text_analysis'] = text_result
            results['modalities_used'].append('text')

        # Analyze image if provided
        if image_path:
            image_result = self.image_analyzer.analyze_image_sentiment(image_path)
            results['image_analysis'] = image_result
            results['modalities_used'].append('image')

        # Combine results if both modalities are present
        if text_result and image_result:
            # Weighted combination
            text_score = self._sentiment_to_score(text_result['sentiment'])
            image_score = self._sentiment_to_score(image_result['sentiment'])

            combined_score = (text_score * text_weight + image_score * image_weight)
            combined_confidence = (text_result['confidence'] * text_weight +
                                 image_result['confidence'] * image_weight)

            results['combined_sentiment'] = self._score_to_sentiment(combined_score)
            results['confidence'] = combined_confidence
            results['combined_score'] = combined_score

        elif text_result:
            results['combined_sentiment'] = text_result['sentiment']
            results['confidence'] = text_result['confidence']

        elif image_result:
            results['combined_sentiment'] = image_result['sentiment']
            results['confidence'] = image_result['confidence']

        return results

    def _sentiment_to_score(self, sentiment):
        """Convert sentiment to numerical score"""
        mapping = {'Negative': -1, 'Neutral': 0, 'Positive': 1}
        return mapping.get(sentiment, 0)

    def _score_to_sentiment(self, score):
        """Convert numerical score to sentiment"""
        if score > 0.2:
            return 'Positive'
        elif score < -0.2:
            return 'Negative'
        else:
            return 'Neutral'

def create_sample_images():
    """
    Create sample images for testing
    """
    sample_images = []

    # Create a bright, colorful image (should be positive)
    img1 = Image.new('RGB', (300, 200), color=(255, 220, 100))  # Bright yellow
    draw1 = ImageDraw.Draw(img1)
    draw1.ellipse([50, 50, 150, 150], fill=(255, 100, 100))  # Red circle
    draw1.ellipse([150, 25, 250, 125], fill=(100, 255, 100))  # Green circle
    sample_images.append(('bright_colors.png', img1, 'Positive'))

    # Create a dark, low contrast image (should be negative)
    img2 = Image.new('RGB', (300, 200), color=(40, 40, 50))  # Dark blue-gray
    draw2 = ImageDraw.Draw(img2)
    draw2.rectangle([50, 50, 250, 150], fill=(30, 30, 40))  # Darker rectangle
    sample_images.append(('dark_image.png', img2, 'Negative'))

    # Create a balanced, neutral image
    img3 = Image.new('RGB', (300, 200), color=(128, 128, 128))  # Gray
    draw3 = ImageDraw.Draw(img3)
    draw3.ellipse([100, 50, 200, 150], fill=(150, 150, 150))  # Light gray circle
    sample_images.append(('neutral_image.png', img3, 'Neutral'))

    # Save sample images
    for filename, img, expected in sample_images:
        img.save(filename)
        print(f"Created sample image: {filename} (expected: {expected})")

    return [filename for filename, _, _ in sample_images]

def demo_comprehensive_image_analysis():
    """
    Comprehensive demo of image sentiment analysis
    """
    print("Comprehensive Image Sentiment Analysis Demo")
    print("-------------------------------------------------------------------------")

    # Create sample images
    print("\nCreating sample images for testing...")
    sample_images = create_sample_images()

    # Initialize analyzer
    analyzer = ImageSentimentAnalyzer()

    print(f"\nAnalyzing {len(sample_images)} sample images:")
    print("-------------------------------------------------------------------------")

    for i, image_path in enumerate(sample_images, 1):
        print(f"\nImage {i}: {image_path}")

        # Analyze image
        result = analyzer.analyze_image_sentiment(image_path)

        print(f"Sentiment: {result['sentiment']}")
        print(f"Confidence: {result['confidence']:.3f}")
        print(f"Score: {result['total_score']:.3f}")

        # Show breakdown
        breakdown = result['analysis_breakdown']
        print("Analysis breakdown:")
        print(f"  Color sentiment: {breakdown['color_analysis']['sentiment_score']:.3f}")
        print(f"  Texture sentiment: {breakdown['texture_analysis']['sentiment_score']:.3f}")
        print(f"  Composition sentiment: {breakdown['composition_analysis']['sentiment_score']:.3f}")

        # Show key features
        color_features = breakdown['color_analysis']['color_features']
        print(f"  Brightness: {color_features['average_brightness']:.3f}")
        print(f"  Saturation: {color_features['average_saturation']:.3f}")
        print(f"  Warmth ratio: {color_features['warmth_ratio']:.3f}")

def demo_multimodal_with_images():
    """
    Demo multimodal analysis with actual images
    """
    print("\nMultimodal Analysis with Images Demo")
    print("-------------------------------------------------------------------------")

    # Create sample images if they don't exist
    if not os.path.exists('bright_colors.png'):
        create_sample_images()

    # Initialize multimodal analyzer
    analyzer = MultimodalSentimentAnalyzer()

    # Test cases with text and image combinations
    test_cases = [
        {
            'text': "What a beautiful sunny day!",
            'image': 'bright_colors.png',
            'description': "Positive text + bright image"
        },
        {
            'text': "I'm feeling really sad today...",
            'image': 'dark_image.png',
            'description': "Negative text + dark image"
        },
        {
            'text': "The weather is okay, nothing special.",
            'image': 'neutral_image.png',
            'description': "Neutral text + neutral image"
        },
        {
            'text': "This is amazing!",
            'image': 'dark_image.png',
            'description': "Positive text + dark image (conflict)"
        }
    ]

    print(f"\nTesting {len(test_cases)} multimodal scenarios:")
    print("-------------------------------------------------------------------------")

    for i, case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {case['description']}")
        print(f"Text: '{case['text']}'")
        print(f"Image: {case['image']}")

        # Analyze with different weight combinations
        result = analyzer.analyze_multimodal_sentiment(
            text=case['text'],
            image_path=case['image'],
            text_weight=0.6,
            image_weight=0.4
        )

        print(f"Combined Sentiment: {result['combined_sentiment']}")
        print(f"Combined Confidence: {result['confidence']:.3f}")

        if 'text_analysis' in result:
            print(f"  Text sentiment: {result['text_analysis']['sentiment']} ({result['text_analysis']['confidence']:.3f})")

        if 'image_analysis' in result:
            print(f"  Image sentiment: {result['image_analysis']['sentiment']} ({result['image_analysis']['confidence']:.3f})")

def analyze_custom_image():
    """
    Allow user to analyze their own image
    """
    print("\nCustom Image Analysis")
    print("-------------------------------------------------------------------------")

    image_path = input("Enter image path or URL (or press Enter to skip): ").strip()

    if not image_path:
        print("Skipping custom image analysis")
        return

    text_input = input("Enter optional text description (or press Enter to skip): ").strip()

    # Initialize analyzer
    analyzer = MultimodalSentimentAnalyzer()

    print(f"\nAnalyzing: {image_path}")
    if text_input:
        print(f"With text: '{text_input}'")

    try:
        if text_input:
            # Multimodal analysis
            result = analyzer.analyze_multimodal_sentiment(
                text=text_input,
                image_path=image_path
            )
            print(f"\nCombined Result:")
            print(f"Sentiment: {result['combined_sentiment']}")
            print(f"Confidence: {result['confidence']:.3f}")

            if 'image_analysis' in result:
                img_result = result['image_analysis']
                print(f"\nImage Analysis Details:")
                print(f"  Sentiment: {img_result['sentiment']}")

        else:
            # Image-only analysis
            img_result = analyzer.image_analyzer.analyze_image_sentiment(image_path)
            print(f"\nImage Analysis Result:")
            print(f"Sentiment: {img_result['sentiment']}")
            print(f"Confidence: {img_result['confidence']:.3f}")

            if 'analysis_breakdown' in img_result:
                print("  Analysis Breakdown:")
                for key, value in img_result['analysis_breakdown'].items():
                    print(f"    {key}: Sentiment Score {value['sentiment_score']:.3f}, Confidence {value['confidence']:.3f}")

    except Exception as e:
        print(f"\nError during analysis: {e}")


def create_sample_data():
    """
    Create sample data for testing
    """
    sample_texts = [
        "I absolutely love this movie! It's fantastic and amazing!",
        "This product is terrible and I regret buying it.",
        "The service was okay, nothing special but not bad either.",
        "Incredible performance! Outstanding work by everyone involved!",
        "Worst experience ever. Completely disappointed and frustrated."
    ]

    return sample_texts

def demo_multimodal_analysis():
    """
    Demonstrate the multimodal sentiment analyzer (text only for simplicity in this version)
    """
    print("Multimodal Sentiment Analysis Demo (Text Only)")
    print("-------------------------------------------------------------------------")

    # Create analyzer
    analyzer = MultimodalSentimentAnalyzer()

    # Test cases
    test_cases = create_sample_data()

    print("\nText Sentiment Analysis:")
    print("-------------------------------------------------------------------------")

    for i, text in enumerate(test_cases, 1):
        result = analyzer.analyze_text_sentiment(text)
        print(f"\nTest {i}:")
        print(f"Text: '{text}'")
        print(f"Sentiment: {result['sentiment']}")
        print(f"Confidence: {result['confidence']:.3f}")
        if 'scores' in result:
            print(f"  Scores: Pos {result['scores']['positive']:.3f}, Neg {result['scores']['negative']:.3f}, Neu {result['scores']['neutral']:.3f}")

    # Example of combined analysis (conceptually, would need image input)
    print("\nConceptual Multimodal Example:")
    print("-------------------------------------------------------------------------")
    print("Analyzing text 'Great picture!' and a hypothetical positive image...")
    conceptual_result = analyzer.analyze_multimodal_sentiment(text="Great picture!", text_weight=0.7, image_weight=0.3)
    print(f"  Combined Sentiment (conceptual): Positive") # Assuming positive image analysis
    print(f"  Combined Confidence (conceptual): High") # Assuming high confidence


def batch_sentiment_analysis():
    """
    Perform batch sentiment analysis on sample data
    """
    print("Batch Sentiment Analysis")
    print("-------------------------------------------------------------------------")

    # Create analyzer
    analyzer = MultimodalSentimentAnalyzer()

    # Get sample data
    texts = create_sample_data()

    results = []
    for i, text in enumerate(texts, 1):
        result = analyzer.analyze_text_sentiment(text)

        results.append({
            'id': i,
            'text': text[:50] + "..." if len(text) > 50 else text,
            'sentiment': result['sentiment'],
            'confidence': result['confidence']
        })

    # Display results
    print(f"\nAnalyzed {len(texts)} texts:")
    print("-------------------------------------------------------------------------")

    for result in results:
        print(f"{result['id']}. {result['text']}")
        print(f"   Sentiment: {result['sentiment']} (Confidence: {result['confidence']:.3f})")
        print()

    # Summary statistics
    sentiments = [r['sentiment'] for r in results]
    positive_count = sentiments.count('Positive')
    negative_count = sentiments.count('Negative')
    neutral_count = sentiments.count('Neutral')

    print("Summary:")
    print(f"Positive: {positive_count}, Negative: {negative_count}, Neutral: {neutral_count}")

if __name__ == "__main__":
    print("Multimodal Sentiment Analysis with Alternative Libraries")
    print("-------------------------------------------------------------------------")

    choice = input("\nChoose demo:\n1. Basic demo (Text Only)\n2. Batch analysis\n3. Comprehensive Image Analysis Demo\n4. Multimodal Analysis with Images Demo\n5. Analyze Custom Image\nEnter choice (1, 2, 3, 4, or 5): ")

    if choice == "2":
        batch_sentiment_analysis()
    elif choice == "3":
        demo_comprehensive_image_analysis()
    elif choice == "4":
        demo_multimodal_with_images()
    elif choice == "5":
        analyze_custom_image()
    else:
        demo_multimodal_analysis()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Multimodal Sentiment Analysis with Alternative Libraries
-------------------------------------------------------------------------

Choose demo:
1. Basic demo (Text Only)
2. Batch analysis
3. Comprehensive Image Analysis Demo
4. Multimodal Analysis with Images Demo
5. Analyze Custom Image
Enter choice (1, 2, 3, 4, or 5): 4

Multimodal Analysis with Images Demo
-------------------------------------------------------------------------


/tmp/ipython-input-945518230.py:50: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  self.image_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')



Testing 4 multimodal scenarios:
-------------------------------------------------------------------------

Test 1: Positive text + bright image
Text: 'What a beautiful sunny day!'
Image: bright_colors.png
Analyzing image: (300, 200)
Combined Sentiment: Positive
Combined Confidence: 0.930
  Text sentiment: Positive (0.950)
  Image sentiment: Positive (0.900)

Test 2: Negative text + dark image
Text: 'I'm feeling really sad today...'
Image: dark_image.png
Analyzing image: (300, 200)
Combined Sentiment: Negative
Combined Confidence: 0.560
  Text sentiment: Negative (0.680)
  Image sentiment: Neutral (0.380)

Test 3: Neutral text + neutral image
Text: 'The weather is okay, nothing special.'
Image: neutral_image.png
Analyzing image: (300, 200)
Combined Sentiment: Positive
Combined Confidence: 0.388
  Text sentiment: Positive (0.447)
  Image sentiment: Neutral (0.300)

Test 4: Positive text + dark image (conflict)
Text: 'This is amazing!'
Image: dark_image.png
Analyzing image: (300, 200)
Co